원천 데이터를 처리하여 가용한 데이터로 변환한다. 처리 내용은 아래와 같다.

1. 분석할 수 없는 파일을 제거한다.

2. sampling rate를 통일한다.

3. state 별 파일을 폴더 이름을 통해 구분한다. 이때 상위 7개의 state에 대해서만 구분하며 나머지는 etc 파일에 csv와 함께 저장한다.


In [5]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('/Users/jaewone/developer/tensorflow/baby-cry-classification')

In [6]:
from utils.os import *
from utils.sound import *
from constant.os import *

In [7]:
origin_data_path = data_path
temp_path = os.path.join(main_path, '_temp')
csv_path = csv_path

if os.path.exists(temp_path):
    remove_path_with_files(temp_path)
os.makedirs(temp_path)

In [8]:
df = pd.read_csv(csv_path, index_col=0)

top7_label = df.state.value_counts().keys().tolist()[:7]
top7_label

['sad', 'hungry', 'hug', 'awake', 'sleepy', 'uncomfortable', 'diaper']

In [9]:
# Sampling rate를 가져오며 분석할 수 없는 파일은 제외한다.
from trans_data import get_sample_rate

# 음성의 sample rate를 추출한 origin_sample_rate 열을 추가한다.
# 에러가 난 파일은 na 값을 가진다.
df = df.assign(origin_sample_rate=df['file'].apply(
    lambda file: get_sample_rate(os.path.join(data_path, file))))

In [10]:
# 분석할 수 없는 데이터는 csv에서 제외한 다음 저장한다.

print(f'전체 {len(df)}개의 데이터 중 사용할 수 없는 {(len(df[df.origin_sample_rate.isna()]))}개의 데이터가 제외되었다.')

df = df[df.origin_sample_rate.notna()]
df.to_csv(csv_path)
df.tail(3)

전체 3688개의 데이터 중 사용할 수 없는 216개의 데이터가 제외되었다.


,state,gender,age,source,file,detail,duration,origin_sample_rate
3685,uncomfortable,NaN,NaN,iFLYTEK,uncomfortable_109.wav,NaN,16.782222,44100.0
3686,uncomfortable,NaN,NaN,iFLYTEK,uncomfortable_135.wav,NaN,13.100567,44100.0
3687,uncomfortable,NaN,NaN,iFLYTEK,uncomfortable_121.wav,NaN,15.530000,16000.0


In [11]:
# Sampling rate를 통일한다.
from bit_sampling import resampling, is_same_sample_rate

resampling(
    file_path_list=[os.path.join(data_path, file)
                    for file in df.file.tolist()],
    output_path=temp_path,
    target_sample_rate=16000
)

if is_same_sample_rate([os.path.join(temp_path, file) for file in os.listdir(temp_path)], 16000):
    print(f'존재하는 sample rate는 16000 뿐이다.')

존재하는 sample rate는 16000 뿐이다.


In [12]:
# 라벨에 따른 폴더 생성
for folder in top7_label:
    os.makedirs(os.path.join(temp_path, folder))
os.makedirs(os.path.join(temp_path, 'etc'))

In [13]:
# 각 라벨에 맞는 폴더로 이동
def move_file_by_label(ser):
    file, state = ser
    move_file(
        os.path.join(temp_path, file),
        os.path.join(temp_path, state, file))


df = df.assign(istop=df.state.where(df.state.isin(top7_label), 'etc'))

df[['file', 'istop']].apply(lambda ser: move_file_by_label(ser), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
3683    None
3684    None
3685    None
3686    None
3687    None
Length: 3472, dtype: object

In [14]:
df[df['istop'] == 'etc'][['state', 'file']].to_csv(os.path.join(temp_path, 'etc.csv'))

In [15]:
df[df['istop'] != 'etc'][['state', 'file']].to_csv(os.path.join(temp_path, 'top7.csv'))

In [16]:
df = df.drop(columns='istop')
df.to_csv(os.path.join(temp_path, 'info.csv'))

미연의 사고를 방지하기 위해 원본 데이터인 data 폴더를 삭제하는 것은 사용자에게 맡긴다.

data 폴더를 삭제한 다음 _temp 폴더의 이름을 data로 변형하여 사용하면 된다.